In [1]:
import collections
import functools

from absl import app

import pandas as pd


import numpy as np
import tensorflow as tf
import tensorflow_federated as tff

from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score
from sklearn.model_selection import train_test_split
import sys

sys.path.append("/home/tester/Desktop/federated/tensorflow_federated/examples/simple_fedavg")
print(sys.path)

import simple_fedavg_tff

2022-06-07 12:22:49.498391: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-06-07 12:22:49.498419: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


['/home/tester/Desktop/federated/tensorflow_federated/examples/simple_fedavg', '/usr/lib/python39.zip', '/usr/lib/python3.9', '/usr/lib/python3.9/lib-dynload', '', '/home/tester/venv/lib/python3.9/site-packages', '/home/tester/Desktop/federated/tensorflow_federated/examples/simple_fedavg']
['/home/tester/Desktop/federated/tensorflow_federated/examples/simple_fedavg', '/usr/lib/python39.zip', '/usr/lib/python3.9', '/usr/lib/python3.9/lib-dynload', '', '/home/tester/venv/lib/python3.9/site-packages', '/home/tester/Desktop/federated/tensorflow_federated/examples/simple_fedavg', '/home/tester/Desktop/federated/tensorflow_federated/examples/simple_fedavg']


In [2]:
# Training hyperparameters
TOTAL_ROUNDS = 50 #NUMBER OF TOTAL TRAINING ROUNDS
ROUNDS_PER_EVAL = 1 #HOW OFTEN TO EVALUATE
TRAIN_CLIENTS_PER_ROUND = 2 #HOW MANY CLIENTS TO SAMPLE PER ROUND
CLIENT_EPOCHS_PER_ROUND = 1 #NUMBER OF EPOCHS IN THE CLIENT TO TAKE PER ROUND
BATCH_SIZE = 16 #BATCH SIZE USED ON THE CLIENT
TEST_BATCH_SIZE = 128 #MINIBATCH SIZE OF TEST DATA
SERVER_LEARNING_RATE = 1.0 #SERVER LEARNING RATE
CLIENT_LEARNING_RATE = 0.1 #CLIENT LEARNING RATE

In [3]:
NUM_CLIENTS = 10
#BATCH_SIZE = 1

In [4]:
dataframe = pd.read_csv("/home/tester/Desktop/TF/datasets/TON_IoT-Datasets/Train_Test_datasets/Train_Test_IoT_dataset/Train_Test_IoT_Modbus.csv")
columns = range(3,8)
df = dataframe.iloc[:, columns]


In [5]:
#NOT REAL, USING ALL THE DATASET - NORMALIZATION MUST BE DONE LOCALLY OR TAKING INTO ACCOUNT CLIENT ROW
df_norm = df.copy()
df_norm = df_norm.iloc[:, range(4)]
for cl in df_norm.columns:
    df_norm[cl].astype('float32')
    min = np.min(df_norm[cl])
    max = np.max(df_norm[cl])
    df_norm[cl] = (df_norm[cl] - min) / (max - min)

df_norm = pd.concat([df_norm, df.iloc[:,-1]], axis=1)


In [6]:
client_id_colname = 'clis'

#IID or NOIID?
choices = np.random.choice(range(NUM_CLIENTS), size=len(df_norm), replace=True)

dcli = {client_id_colname: choices}
dfcli = pd.DataFrame(data=dcli)

df = pd.concat([df_norm, dfcli], axis=1)

In [7]:

#df = pd.read_csv(csv_url, na_values=("?",))

SHUFFLE_BUFFER = 1000
NUM_EPOCHS = 1
# split client id into train and test clients
client_ids = df[client_id_colname].unique()
train_client_ids = pd.DataFrame(client_ids).sample(frac=0.8).values.tolist()
test_client_ids = [[x] for x in client_ids if x not in train_client_ids]

def create_tf_dataset_for_client_fn(client_id):
  # a function which takes a client_id and returns a
  # tf.data.Dataset for that client
  client_data = df[df[client_id_colname] == client_id[0]]
  dataset = tf.data.Dataset.from_tensor_slices(client_data.fillna('').to_dict("list"))
  dataset = dataset.shuffle(SHUFFLE_BUFFER).batch(BATCH_SIZE).repeat(NUM_EPOCHS)
  return dataset

train_data = tff.simulation.datasets.ClientData.from_clients_and_tf_fn(
        client_ids=train_client_ids,
        serializable_dataset_fn = create_tf_dataset_for_client_fn
        #create_tf_dataset_for_client_fn=create_tf_dataset_for_client_fn
    )

test_data = tff.simulation.datasets.ClientData.from_clients_and_tf_fn(
         client_ids=test_client_ids,
         serializable_dataset_fn = create_tf_dataset_for_client_fn
         #create_tf_dataset_for_client_fn=create_tf_dataset_for_client_fn
     )

example_dataset = train_data.create_tf_dataset_for_client(
        train_data.client_ids[0]
    )

print(type(example_dataset))
example_element = iter(example_dataset).next()
print(example_element)


<class 'tensorflow.python.data.ops.dataset_ops.RepeatDataset'>
{'FC1_Read_Input_Register': <tf.Tensor: shape=(16,), dtype=float32, numpy=
array([0.002457  , 0.68028444, 0.8921666 , 0.21311826, 0.24531873,
       0.581165  , 0.5106292 , 0.367711  , 0.9392464 , 0.8189754 ,
       0.06577441, 0.02347124, 0.40429136, 0.94177973, 0.8336106 ,
       0.3244922 ], dtype=float32)>, 'FC2_Read_Discrete_Value': <tf.Tensor: shape=(16,), dtype=float32, numpy=
array([0.33917755, 0.6746624 , 0.22465858, 0.6243839 , 0.13745327,
       0.09192035, 0.19319448, 0.50479895, 0.7345083 , 0.4886244 ,
       0.93846035, 0.41162738, 0.06213474, 0.8530251 , 0.24805066,
       0.46640727], dtype=float32)>, 'FC3_Read_Holding_Register': <tf.Tensor: shape=(16,), dtype=float32, numpy=
array([0.57153755, 0.34205264, 0.7459901 , 0.29558185, 0.86257154,
       0.3044029 , 0.779794  , 0.8117207 , 0.17210226, 0.155971  ,
       0.6734987 , 0.15325448, 0.02519649, 0.5322091 , 0.38866082,
       0.11296452], dtype=float32)>

2022-06-07 12:22:51.820986: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-06-07 12:22:51.821024: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-06-07 12:22:51.821041: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (debian): /proc/driver/nvidia/version does not exist
2022-06-07 12:22:51.821393: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [8]:
def preprocess(dataset):
  def batch_format_fn(element):
    """Flatten a batch of EMNIST data and return a (features, label) tuple."""
    return (tf.reshape([element['FC1_Read_Input_Register'], element['FC2_Read_Discrete_Value'], element['FC3_Read_Holding_Register'], element['FC4_Read_Coil']], [-1, 4]),
            tf.reshape(element['label'], [-1, 1]))
  return dataset.batch(BATCH_SIZE).map(batch_format_fn)

In [9]:
client_ids = sorted(train_data.client_ids)[:NUM_CLIENTS]
federated_train_data = [preprocess(train_data.create_tf_dataset_for_client(x))
  for x in client_ids
]

In [10]:
client_ids = sorted(test_data.client_ids)[:NUM_CLIENTS]
federated_test_data = [preprocess(test_data.create_tf_dataset_for_client(x))
  for x in client_ids
]

In [ ]:
def get_subset_of_dataset (data, client_ids):
    data[0]

In [11]:
def evaluate(keras_model, test_dataset):
  """Evaluate the acurracy of a keras model on a test dataset."""
  metric = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')
  for batch in test_dataset:
    predictions = keras_model(batch['x'])
    metric.update_state(y_true=batch['y'], y_pred=predictions)
  return metric.result()


def get_custom_dataset():
  return federated_train_data, federated_test_data


def create_fedavg_model():
  initializer = tf.keras.initializers.GlorotNormal(seed=0)
  return tf.keras.models.Sequential([
    tf.keras.layers.Input(shape=(4,)),
    tf.keras.layers.Dense(8),
    tf.keras.layers.Dense(10),
    tf.keras.layers.Dense(8),
    #tf.keras.layers.Dense(10, kernel_initializer=initializer),
    tf.keras.layers.Softmax()
    ])


def server_optimizer_fn():
  return tf.keras.optimizers.SGD(learning_rate=SERVER_LEARNING_RATE)


def client_optimizer_fn():
  return tf.keras.optimizers.SGD(learning_rate=CLIENT_LEARNING_RATE)

In [12]:
def main(argv):
  if len(argv) > 1:
    raise app.UsageError('Too many command-line arguments.')
  # If GPU is provided, TFF will by default use the first GPU like TF. The
  # following lines will configure TFF to use multi-GPUs and distribute client
  # computation on the GPUs. Note that we put server computatoin on CPU to avoid
  # potential out of memory issue when a large number of clients is sampled per
  # round. The client devices below can be an empty list when no GPU could be
  # detected by TF.
  client_devices = tf.config.list_logical_devices('GPU')
  server_device = tf.config.list_logical_devices('CPU')[0]
  tff.backends.native.set_local_python_execution_context(
      server_tf_device=server_device, client_tf_devices=client_devices)
  train_data, test_data = get_custom_dataset()

  def tff_model_fn():
    """Constructs a fully initialized model for use in federated averaging."""
    keras_model = create_fedavg_model()
    loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
    metrics = [tf.keras.metrics.SparseCategoricalAccuracy()]
    return tff.learning.from_keras_model(
        keras_model,
        input_spec=federated_train_data[0].element_spec,
        loss=loss,
        metrics=metrics)

  iterative_process = simple_fedavg_tff.build_federated_averaging_process(
      tff_model_fn, server_optimizer_fn, client_optimizer_fn)
  server_state = iterative_process.initialize()
  # Keras model that represents the global model we'll evaluate test data on.
  keras_model = create_fedavg_model()
  for round_num in range(TOTAL_ROUNDS):
    sampled_clients = np.random.choice(
        range(NUM_CLIENTS),
        size=TRAIN_CLIENTS_PER_ROUND,
        replace=False)
    
    sampled_train_data = create_tf_dataset_for_client_fn()
    tff.simulation.datasets.ClientData.from_clients_and_tf_fn(
        client_ids=sampled_clients,
        serializable_dataset_fn = create_tf_dataset_for_client_fn
        #create_tf_dataset_for_client_fn=create_tf_dataset_for_client_fn
    )
    
    sampled_train_data = [
        federated_train_data.create_tf_dataset_for_client(client)
        for client in sampled_clients
    ]
    server_state, train_metrics = iterative_process.next(
        server_state, sampled_train_data)
    print(f'Round {round_num}')
    print(f'\tTraining metrics: {train_metrics}')
    if round_num % ROUNDS_PER_EVAL == 0:
      server_state.model.assign_weights_to(keras_model)
      accuracy = evaluate(keras_model, test_data)
      print(f'\tValidation accuracy: {accuracy * 100.0:.2f}%')

Patch asyncio to allow nested event loops. If not used, runtime error in Jupyter NB

In [13]:
import nest_asyncio
nest_asyncio.apply()

In [14]:
app.run(main([None]))

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [ ]:
try:
    app.run(main([None]))
except:
    print("exec end error")

In [15]:
tt = tff.simulation.datasets.TestClientData(
    client_ids
)


TypeError: Expected dict, found list.